In [1]:
from pathlib import Path
import shutil
from tqdm import tqdm
from useYolo import get_xyxy as detect_img
from Detector_show4d import Detector
import model as qrcnn_model
from glob import glob
import os

Using cuda device
log 存檔路徑訂為 : D:\Git\zjpj\log_save\2022-08-28_AM_02h43m43s


In [10]:
def xyxyBox_IOU(boxA, boxB):
    # 使用 xyxy 座標，單位統一即可
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # 計算兩個正方形的交集面積
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # 分別計算兩個正方形的面積
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # 交集 / 聯集
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [54]:
# 基於用 lagelimg 標記好的 txt 檔名，去將原始圖片目錄下的東西 複製到 新的資料夾

base_root = r"D:\Git\zjpj\data_clean\yolo_label_NOT_IN_the_real593_DATASET"  # 所需的標記檔案 txt
image_src = r"D:\Git\zjpj\data_clean\NOT_IN_the_real593_DATASET" # 圖片目錄，檔案數量會 >= base_root
dst_path = r"D:\Git\zjpj\data_clean\IoU_test_image"  # 要將圖片複製到的目的

image_stem = [ Path(_).stem for _ in glob(image_src+'/*.*')]
labels = glob(base_root+'/*.*')
labels.pop() # remove last one.

for _ in labels:
    label_stem = Path(_).stem
    if label_stem in image_stem:
        # cpoy image
        shutil.copy2(Path(image_src).joinpath(f"{label_stem}.png"),
                    Path(dst_path).joinpath(f"{label_stem}.png"))
        # copy label
        # shutil.copy2(Path(label_src).joinpath(f"{label_stem}.txt"),
        #             Path(dst_path).joinpath(f"{label_stem}.txt"))

In [9]:
def get_ground_truth_norm_bbox(image_path, label_root=r"D:\Git\zjpj\data_clean\yolo_label_NOT_IN_the_real593_DATASET"):
    label_path = Path(label_root).joinpath(Path(image_path).stem+'.txt')
    assert os.path.isfile(label_path)
    bbox = []
    with open(label_path, mode='r') as f:
        contents = [_.rstrip()for _ in f.readlines()]
        
        for i in contents:
            _,x,y,w,h = [float(_) for _ in i.split(' ')]
            x1 = x-(w/2)
            y1 = y-(h/2)
            x2 = x+(w/2)
            y2 = y+(h/2)
            bbox.append((x1,y1,x2,y2))
    
    return bbox
#
get_ground_truth_norm_bbox(r"D:\Git\zjpj\data_clean\IoU_test_image\00053.png")

[(0.07446749999999999, 0.16763, 0.2411345, 0.442196),
 (0.41844, 0.173411, 0.58688, 0.445087),
 (0.760638, 0.16763, 0.932624, 0.442196)]

In [3]:
IOU_estm_root = r'D:\Git\zjpj\data_clean\IoU_test_image'

# 初始化舊的模型
old_detector = Detector(weight="./50weight.pt", save_folder="./JUPYTER_LAB_TMP_LOG",
             net=qrcnn_model.QRCode_CNN())

In [23]:
label_root=r"D:\Git\zjpj\data_clean\yolo_label_NOT_IN_the_real593_DATASET"

old_cnt = 0
bad_image = []

pbar = tqdm(glob(IOU_estm_root+'/*.*'), ncols=110)
bbox_result = []
for idx, image_path in enumerate(pbar):
    pbar.set_description(f"current:{image_path}")
    new_bbox = [res[0:4] for res in detect_img(image_path,norm=True)]  # 取 0,1,2,3 ， 4,5 是 pres and classes
    old_bbox = old_detector.detect_nrom_bbox(image_path)
    gdt_bbox = get_ground_truth_norm_bbox(image_path, label_root)
    print(new_bbox)
    print(old_bbox)
    print(gdt_bbox)
    
    
    n_iou = []
    o_iou = []
    for gx1,gy1,gx2,gy2 in gdt_bbox:  # 全部的 GT bbox
        
        n_max_IOU = 0.0
        for nx1,ny1,nx2,ny2 in new_bbox:  # 與 強化過模型的 bbox 做 iou比較
            iou = old_detector.xyxyBox_IOU((nx1,ny1,nx2,ny2),(gx1,gy1,gx2,gy2))
            if iou > n_max_IOU:
                n_max_IOU = iou
        n_iou.append(n_max_IOU)
        
        o_max_IOU = 0.0
        for ox1,oy1,ox2,oy2 in old_bbox:  # 與 舊模型的 bbox 做 iou比較
            iou = old_detector.xyxyBox_IOU((ox1,oy1,ox2,oy2),(gx1,gy1,gx2,gy2))
            if iou > o_max_IOU:
                o_max_IOU = iou
        o_iou.append(o_max_IOU)
    
    print("avg NEW iou:", sum(n_iou)/len(n_iou))
    print("avg OLD iou:", sum(o_iou)/len(o_iou))
    if sum(o_iou)/len(o_iou) == 0.0:
        bad_image.append(image_path)
    elif len(new_bbox) == 0:
        bad_image.append(image_path)
    else:
        print("===============================================")
print("cnt:", old_cnt)
print("bad name:", bad_image)
    

current:D:\Git\zjpj\data_clean\IoU_test_image\00005.png:   0%|                         | 0/95 [00:00<?, ?it/s]

[(0.37666666666666665, 0.396969696969697, 0.62, 0.6848484848484848)]
[(0.36, 0.4008097165991903, 0.7022222222222222, 0.7125506072874493)]
[(0.37666700000000003, 0.409091, 0.6166670000000001, 0.693939)]
avg NEW iou: 0.9810264047609132
avg OLD iou: 0.9049007667402502


current:D:\Git\zjpj\data_clean\IoU_test_image\00008.png:   2%|▎                | 2/95 [00:00<00:20,  4.56it/s]

[(0.2725, 0.49333333333333335, 0.7675, 0.8233333333333334)]
[(0.36, 0.14666666666666667, 0.4911111111111111, 0.22074074074074074), (0.8, 0.14666666666666667, 0.9111111111111111, 0.22074074074074074), (0.02, 0.6266666666666667, 0.09111111111111111, 0.674074074074074), (0.0, 0.72, 0.07111111111111111, 0.7674074074074074), (0.6, 0.76, 0.7311111111111112, 0.8474074074074074), (0.0, 0.7866666666666666, 0.07111111111111111, 0.8340740740740741)]
[(0.2475, 0.4783335, 0.7925, 0.8416665000000001)]
avg NEW iou: 0.9439791516838434
avg OLD iou: 0.5790704491714919
[(0.14, 0.10285714285714286, 0.8828571428571429, 0.88)]
[(0.10305343511450382, 0.13740458015267176, 0.7404580152671756, 0.8778625954198473)]
[(0.09428550000000002, 0.08571450000000003, 0.9114285, 0.8971435)]
avg NEW iou: 0.9409655189256481
avg OLD iou: 0.8657831524531969


current:D:\Git\zjpj\data_clean\IoU_test_image\00012.png:   4%|▋                | 4/95 [00:00<00:14,  6.34it/s]

...

[(0.2075, 0.3949367088607595, 0.755, 0.9493670886075949)]
[(0.21, 0.3952702702702703, 0.7666666666666667, 0.9594594594594594), (0.0, 0.7297297297297297, 0.10666666666666667, 0.8378378378378378)]
[(0.21249999999999997, 0.392405, 0.7625, 0.946835)]
avg NEW iou: 0.9887481268374534
avg OLD iou: 0.9858827823800311


current:D:\Git\zjpj\data_clean\IoU_test_image\01502.png:  98%|███████████████▋| 93/95 [00:15<00:00,  7.71it/s]

[(0.2816666666666667, 0.19166666666666668, 0.3566666666666667, 0.2683333333333333), (0.7466666666666667, 0.6716666666666666, 0.8216666666666667, 0.7466666666666667)]
[(0.7, 0.6, 0.8511111111111112, 0.7711111111111111)]
[(0.745, 0.6700005, 0.8250000000000001, 0.7533335000000001), (0.28, 0.19166650000000002, 0.36, 0.2749995)]
avg NEW iou: 0.988479648876929
avg OLD iou: 0.5414254065801719
[(0.64, 0.7694235588972431, 0.6983333333333334, 0.8546365914786967)]
[(0.0, 0.2709030100334448, 0.1111111111111111, 0.4682274247491639), (0.18, 0.2709030100334448, 0.2511111111111111, 0.3779264214046823)]
[(0.64, 0.7694234999999999, 0.7000000000000001, 0.8571425), (0.4316665, 0.7844615, 0.4949995, 0.8872185)]
avg NEW iou: 0.8234277826332674
avg OLD iou: 0.22442066554878576


current:D:\Git\zjpj\data_clean\IoU_test_image\01505.png: 100%|████████████████| 95/95 [00:15<00:00,  6.22it/s]

[(0.0, 0.0, 0.992822966507177, 0.95), (0.8325358851674641, 0.8366666666666667, 0.9617224880382775, 0.9266666666666666), (0.6818181818181818, 0.8333333333333334, 0.8086124401913876, 0.9266666666666666)]
[(0.5, 0.8232558139534883, 0.6711111111111111, 0.9426356589147287), (0.7, 0.8372093023255814, 0.8111111111111111, 0.9286821705426357), (0.82, 0.8372093023255814, 0.9111111111111111, 0.9007751937984496)]
[(0.528708, 0.833333, 0.669856, 0.928333), (0.6842105, 0.8316665, 0.8157895, 0.9299995), (0.8349285000000001, 0.8366665, 0.9641155, 0.9283334999999999)]
avg NEW iou: 0.9174083065422964
avg OLD iou: 0.9485634496080427
[(0.15, 0.6582278481012658, 0.21166666666666667, 0.8227848101265823)]
[(0.56, 0.711864406779661, 0.6311111111111111, 0.8926553672316384)]
[(0.148333, 0.6624475, 0.213333, 0.8270045)]
avg NEW iou: 0.9896832302145712
avg OLD iou: 0.41011922938180284
cnt: 0
bad name: []


In [53]:
old_bbox = old_detector.detect_bbox(img_path)